## 1. 데이터 로딩

scikit-learn (sklearn) 은 다양한 머신러닝 알고리즘 및 데이터 처리 모듈들을 가지고 있는 툴킷입니다. sklearn 의 CountVectorizer 를 이용하여 term frequency matrix 를 만드는 법을 연습합니다. 

이를 위해서 10 개의 영화 리뷰 데이터를 모아둔 파일을 이용하겠습니다. 

In [1]:
rawtext_fname = '../../../tutorial_data/lalaland_comments.txt'

파일이 어떤 형식으로 저장되어 있는지 확인하기 위하여 맨 앞 5줄만 출력해봅니다. 

In [2]:
with open(rawtext_fname, encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다	9
사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두말할것없고	10
지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두ㅜㅜ최고입니다	10
방금 시사회 보고 왔어요 배우들 매력이 눈을 뗄 수가 없게 만드네요 한편의 그림 같은 장면들도 많고 음악과 춤이 눈과 귀를 사로 잡았어요 한번 더 보고 싶네요	10
초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음 힐링 받는 느낌들다가 막판에 기분 잡쳤습니다 마치 감독이 하고싶은 말은 너희들이 원하는 결말은 이거지 하지만 현실은 이거다 라고 말하고 싶었나보군요	1


데이터는 <텍스트, 평점> 이 tap 으로 구분되어 있습니다. reveiw 를 로딩하는 함수를 만들어봅니다. 테스트를 위하여 앞의 1,000 줄의 리뷰만 가져옵니다.

데이터를 로딩할 수 있는 함수는 데이터 폴더의 data_loader.py 안에 넣어뒀습니다.

In [3]:
import sys
sys.path.append('../../../tutorial_data/')

from data_loader import get_lalaland_comments

texts, scores = get_lalaland_comments(n_limit=1000)

## 2. From corpus to noun term frequency sparse matrix

Twitter.nouns(sent) 는 주어진 sent 에서 명사를 추출하여 return 합니다. type 은 list of str 입니다. 

In [4]:
from collections import defaultdict
from pprint import pprint
from konlpy.tag import Komoran

komoran = Komoran()
komoran.nouns('이건 테스트입니다')

['이건', '테스트']

collections.Counter 를 이용하여 한 번에 명사의 개수를 세어 봅시다. 

In [5]:
from collections import Counter

noun_counter = Counter(
    [noun for text in texts for noun in komoran.nouns(text)]
)

print('num of nouns: %d' % len(noun_counter))

num of nouns: 1291


term frequency matrix 를 만들 때, 거의 등장하지 않는 단어들은 의미가 없습니다. min count 를 설정하여 각 threshold 별로 몇 개의 명사가 살아남는지 알아봅니다. 

In [6]:
for min_count in [2, 3, 5, 10]:
    _counter = {
        word for word, freq in noun_counter.items()
        if freq >= min_count
    }
    print('num of nouns (min_count = %d): %d' % (
        min_count, len(_counter)))

num of nouns (min_count = 2): 572
num of nouns (min_count = 3): 375
num of nouns (min_count = 5): 234
num of nouns (min_count = 10): 118


우리는 2번 이상 나오는 572 개의 명사를 이용하여 term frequency matrix 를 만들어 봅니다. 우리는 전체 문서에서의 빈도수가 2 이상인 명사만을 return 하는 토크나이저를 만들 것입니다.

Komoran.nouns() 와 custom_tokenizer의 결과가 달라짐을 확인할 수 있습니다.

In [7]:
noun_dict = {
    word for word, freq in noun_counter.items()
    if freq >= 2
}

def custom_tokenizer(doc):
    return [word for word in komoran.nouns(doc) if word in noun_dict]

print(komoran.nouns(texts[1]))
print(custom_tokenizer(texts[1]))

['사랑', '꿈', '흐름', '음악', '영상', '최대한', '배우', '연기', '두말', '것']
['사랑', '꿈', '흐름', '음악', '영상', '배우', '연기', '것']


In [8]:
len(noun_dict)

572

CountVectorizer 는 document frequency 를 기준으로 대부분 문서에 등장하거나 거의 등장하지 않는 극단적인 단어들을 제거할 수 있습니다. min_df 와 max_df 를  이용하면 되며, df 는 비율입니다. [0, 1] 사이의 값을 입력해야 합니다. 

CountVectorizer 의 argument tokenizer 는 str 형식의 문장이 들어왔을 때 이를 list of str 형식의 단어열로 변형하는 함수입니다. 기본값은 lambda x:x.split()으로 띄어쓰기 기준으로 단어를 나눕니다. 우리가 만든 custom_tokenizer 를 이용하여 빈도수가 2 이상인 명사들만을 return 하는 토크나이저로 이를 대채합니다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    tokenizer=custom_tokenizer,
    min_df=0.005,
    max_df=0.95
)

x_sparse = vectorizer.fit_transform(texts)

x_sparse 는 scipy 의 sparse matrix 입니다. Sparse matrix 는 0 값이 많은 행렬에 대하여 0 이 아닌 (i, j) 의 값만을 저장한 matrix 를 의미합니다. Term frequency matrix 는 very sparse matrix 에 속하므로 dense matrix 보다 sparse matrix 가 훨씬 효율적입니다. dense matrix 는 문서의 양이 조금만 커도 매우 큰 메모리가 필요합니다.

In [10]:
type(x_sparse)

scipy.sparse.csr.csr_matrix

x_sparse 는 1,000 개의 리뷰가 223 개의 단어로 표현된 형태입니다. 이는 우리가 min_df = 0.005 로 설정하였기 때문입니다. Document frequency 가 4 이하인 단어들도 제거되었기 때문입니다.

또한 이 sparse matrix 는 CSR matrix 형식입니다. sparse matrix 의 형식은 이후에 다시 설명하겠습니다.

In [11]:
print(type(x_sparse))
x_sparse

<class 'scipy.sparse.csr.csr_matrix'>


<1000x223 sparse matrix of type '<class 'numpy.int64'>'
	with 4499 stored elements in Compressed Sparse Row format>

## 3. CountVectorizer 에서 각 column 에 해당하는 term 알아내기

vectorizer.vocabulary_ 는 dict 형식으로, {word:index} 입니다. 

    {'가장': 0,
     '가족': 1,
     '가치': 2,
     '각본': 3,
     '간다': 4,
     '감': 5,
     '감독': 6,
     ...
    }

이를 이용하여 vocab to index, index to vocab 을 만들어 봅니다. 영화라는 단어가 115 에 해당합니다. 

In [12]:
vocab2int = vectorizer.vocabulary_
vocab2int['영화']

134

index 는 0 부터 시작합니다. 그러므로 int2vocab 은 dictionary 형태보다 list 형태로 가지고 있어도 좋습니다. 이를 위해 sorted 함수를 이용합니다. 115 번째 단어가 '영화'임을 다시 확인할 수 있습니다. 

In [13]:
int2vocab = [
    word for word,index in sorted(
        vocab2int.items(), key=lambda x:x[1])
]
int2vocab[134]

'영화'

## 4. Sparse matrix I/O

scipy.io 는 sparse matrix 를 읽고 쓰는 기능을 제공합니다. mmwrite 는 sparse matrix 를 filepath 에 저장합니다. 그런데 filepath 의 경로에 directory 가 존재하지 않는다면 오류가 발생합니다. 이를 방지하기 위해서 path 가 존재하는지 확인하고, directory 가 존재하지 않을 경우 이를 만들어야 합니다.

    if not os.path.exists('tmp/'):
        os.makedirs('tmp')

위의 코드는 tmp 폴더가 있는지 확인하고, 존재하지 않는 경우 tmp 폴더를 만드는 코드입니다. 

In [14]:
import os
from scipy.io import mmwrite, mmread

if not os.path.exists('tmp/'):
    os.makedirs('tmp')

mmwrite('./tmp/x.mm', x_sparse)

mmread 는 filepath 의 sparse matrix 를 읽습니다. 그런데 mmread 의 형식은 COO matrix 입니다. 이를 CSR matrix 로 바꾸기 위해 tocsr 함수를 이용합니다.

In [15]:
loaded_x_sparse = mmread('./tmp/x.mm').tocsr()
loaded_x_sparse

<1000x223 sparse matrix of type '<class 'numpy.int64'>'
	with 4499 stored elements in Compressed Sparse Row format>

## 5. Save vectorizer and vocabulary

vectorizer 는 pickling 으로 저장하여도 좋습니다. vocabulary list 는 따로 list 파일로 저장해도 좋습니다. 

In [16]:
import pickle

with open('./tmp/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('./tmp/vocabulary.txt', 'w', encoding='utf-8') as f:
    for vocab in int2vocab:
        f.write('%s\n' % vocab)

In [17]:
with open('./tmp/vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

print(loaded_vectorizer.vocabulary_['영화'])

134


In [18]:
with open('./tmp/vocabulary.txt', encoding='utf-8') as f:
    for _ in range(5):
        print(next(f).strip())

10분
가슴
감
감독
감동
